In [ ]:
import pandas as pd
import ast
import datetime
pd.set_option('display.max_row', 1000)
data = pd.read_csv("/Users/l.sergeev/Downloads/2733570.activities.0de556420a991448bdae453d16cc9ead.0.csv")
data.info()

In [ ]:
#ОСНОВНОЙ ДАТА ФРЕЙМ СО ВСЕМИ ЮЗЕРАМИ НОВОЙ ВЕРСИИ ПРИЛОЖЕНИЯ ПО ДНЯМ

In [ ]:
new_version_users = data.loc[data.idAppVersionTitle.str.contains("5.")].reset_index(drop=True)

In [ ]:
df_users = new_version_users[['customUserId','dtEvent']].drop_duplicates().dropna().reset_index(drop=True)

In [ ]:
df_users.info()

In [ ]:
#ДАТА ФРЕЙМ СО ВСЕМИ PLAY_TRACKS НОВОЙ ВЕРСИИ

In [ ]:
data_tracks = pd.read_csv("/Users/l.sergeev/Downloads/2735459.customEvents.885c3543a96a86cf74c314f4cca2ea64.0.csv")

In [ ]:
cat = pd.read_excel("/Users/l.sergeev/Downloads/cat.xlsx",sheet_name = 'Sheet1')

In [ ]:
new_version_users_tracks = data_tracks.loc[data_tracks.idAppVersionTitle.str.contains("5.")].reset_index(drop=True)

In [ ]:
def event_parser(df):
    names = df['params.name'][0]
    names = ast.literal_eval(names)
    df['params.value'] = df['params.value'].astype(str).apply(lambda x: ast.literal_eval(x))
    df1 = pd.DataFrame(df["params.value"].to_list(), columns=names)
    df2 = df.join(df1)
    return df2

In [ ]:
new_version_users_tracks_parsed = event_parser(new_version_users_tracks)

In [ ]:
new_version_users_tracks_parsed_cat = new_version_users_tracks_parsed.merge(cat,on='from',how='left')

In [ ]:
new_version_users_tracks_parsed_cat.info()

In [ ]:
#ЮЗЕРЫ, КОТОРЫЕ СЛУШАЛИ ММ

In [ ]:
df_users_my_music = new_version_users_tracks_parsed_cat.groupby(['customUserId','dtEvent','normal_title'])['countCustomEvent'].sum().reset_index()
df_users_my_music.head()


In [ ]:
df_users_totals = new_version_users_tracks_parsed_cat.groupby(['customUserId','dtEvent'])['countCustomEvent'].sum().reset_index()



In [ ]:
df_users_my_music_w_totals = df_users_my_music.merge(df_users_totals,on=['dtEvent','customUserId'],how='left')
df_users_my_music_w_totals

In [ ]:
df_users_my_music_w_totals['percent'] = (df_users_my_music_w_totals['countCustomEvent_x'] / df_users_my_music_w_totals['countCustomEvent_y'] * 100).round(2)


In [ ]:
df_heavy_mm_users = df_users_my_music_w_totals[["dtEvent","customUserId",'normal_title','percent']][df_users_my_music_w_totals['normal_title'] == 'Моя Музыка']
df_heavy_mm_users.head()



In [ ]:
df_heavy_mm_users_70 = df_heavy_mm_users[["customUserId",'dtEvent']][df_heavy_mm_users['percent'] >= 70].drop_duplicates().dropna().reset_index(drop=True)
df_heavy_mm_users_70['Heavy_my_music'] = 'Heavy_my_music'

df_heavy_mm_users_70


In [ ]:
df_users_labels = df_users.merge(df_heavy_mm_users_70,on=['dtEvent','customUserId'],how='left')
df_users_labels['Heavy_my_music'] = df_users_labels['Heavy_my_music'].fillna('Not')

df_users_labels

In [ ]:
#ЮЗЕРЫ, КОТОРЫЕ СЛУШАЛИ МИКС

In [ ]:
df__mix_users = df_users_my_music_w_totals[["dtEvent","customUserId"]][df_users_my_music_w_totals['normal_title'] == 'Микс']
df__mix_users = df__mix_users.drop_duplicates().dropna().reset_index(drop=True)
df__mix_users['Mix_users'] = 'Mix_users'

df__mix_users


In [ ]:
df_users_labels = df_users_labels.merge(df__mix_users,on=['dtEvent','customUserId'],how='left')
df_users_labels['Mix_users'] = df_users_labels['Mix_users'].fillna('Not')


In [ ]:
df_users_labels.head(20)

In [ ]:
#df_users_my_music_w_totals.loc[df_users_my_music_w_totals['customUserId'] == 'vk164934334']

In [ ]:
#df_users_labels.loc[df_users_labels['customUserId'] == 'vk164934334']

In [ ]:
#СТРИММИНГ

In [ ]:
df_users_streaming = new_version_users_tracks_parsed_cat.groupby(['customUserId','dtEvent','method'])['countCustomEvent'].sum().reset_index()


In [ ]:
df_users_streaming.head()

In [ ]:
df_users_streaming_totals = new_version_users_tracks_parsed_cat.groupby(['customUserId','dtEvent'])['countCustomEvent'].sum().reset_index()


In [ ]:
df_users_streaming_w_totals = df_users_streaming.merge(df_users_streaming_totals,on=['dtEvent','customUserId'],how='left')
df_users_streaming_w_totals

In [ ]:
df_users_streaming_w_totals['percent'] = (df_users_streaming_w_totals['countCustomEvent_x'] / df_users_streaming_w_totals['countCustomEvent_y'] * 100).round(2)
df_users_streaming_w_totals


In [ ]:
df_users_streaming_online = df_users_streaming_w_totals[["dtEvent","customUserId",'method','percent']][df_users_streaming_w_totals['method'] == 'online']


In [ ]:
df_users_streaming_online_70 = df_users_streaming_online[["customUserId",'dtEvent']][df_users_streaming_online['percent'] >= 70].drop_duplicates().dropna().reset_index(drop=True)
df_users_streaming_online_70['Only_streaming_70'] = 'Only_streaming_70'

df_users_streaming_online_70


In [ ]:
df_users_labels = df_users_labels.merge(df_users_streaming_online_70,on=['dtEvent','customUserId'],how='left')
df_users_labels['Only_streaming_70'] = df_users_labels['Only_streaming_70'].fillna('Not')
df_users_labels

In [ ]:
df_users_streaming_online_10 = df_users_streaming_online[["customUserId",'dtEvent']][df_users_streaming_online['percent'] >= 10].drop_duplicates().dropna().reset_index(drop=True)
df_users_streaming_online_10['Only_streaming_10'] = 'Only_streaming_10'

df_users_streaming_online_10

In [ ]:
df_users_labels = df_users_labels.merge(df_users_streaming_online_10,on=['dtEvent','customUserId'],how='left')
df_users_labels['Only_streaming_10'] = df_users_labels['Only_streaming_10'].fillna('Not')
df_users_labels

In [ ]:
#КЭШ

In [ ]:
df_users_streaming_cache = df_users_streaming_w_totals[["dtEvent","customUserId",'method','percent']][df_users_streaming_w_totals['method'] == 'cache']


In [ ]:
df_users_cache = df_users_streaming_cache[["customUserId",'dtEvent']][df_users_streaming_cache['percent'] >= 50].drop_duplicates().dropna().reset_index(drop=True)
df_users_cache['Cache_50_new'] = 'Cache_50_new'


In [ ]:
df_users_labels = df_users_labels.merge(df_users_cache,on=['dtEvent','customUserId'],how='left')
df_users_labels['Cache_50_new'] = df_users_labels['Cache_50_new'].fillna('Not')
df_users_labels

In [ ]:
#ДОБАВЛЕНИЯ В ММ

In [ ]:
data_add_to_mm = pd.read_csv("/Users/l.sergeev/Downloads/2732690.customEvents.29814639a0448edfe41560c2240b8508.0.csv")

In [ ]:
new_version_users_add_to_mm = data_add_to_mm.loc[data_add_to_mm.idAppVersionTitle.str.contains("5.")].reset_index(drop=True)
new_version_users_add_to_mm



In [ ]:
new_version_users_add_to_mm_parsed = event_parser(new_version_users_add_to_mm)

In [ ]:
new_version_users_add_to_mm_parsed_cat = new_version_users_add_to_mm_parsed.merge(cat,on='from',how='left')

In [ ]:
new_version_users_add_to_mm_parsed_cat

In [ ]:
#ЮЗЕРЫ, КОТОРЫЕ ДОБАВЛЯЛИ ИЗ МИКСА

In [ ]:
df__mix_add_to_mm = new_version_users_add_to_mm_parsed_cat[["dtEvent","customUserId"]][new_version_users_add_to_mm_parsed_cat['normal_title'] == 'Микс']
df__mix_add_to_mm = df__mix_add_to_mm.drop_duplicates().dropna().reset_index(drop=True)
df__mix_add_to_mm['Add_from_mix'] = 'Add_from_mix'

df__mix_add_to_mm

In [ ]:
df_users_labels = df_users_labels.merge(df__mix_add_to_mm,on=['dtEvent','customUserId'],how='left')
df_users_labels['Add_from_mix'] = df_users_labels['Add_from_mix'].fillna('Not')
df_users_labels

In [ ]:
#new_version_users_add_to_mm_parsed_cat.loc[new_version_users_add_to_mm_parsed_cat['customUserId'] == '165883608']

In [ ]:
#ПОДПИСКА

In [ ]:
df__users_subscription = new_version_users_tracks_parsed_cat[["dtEvent","customUserId",'subscription_type']]
df__users_subscription = df__users_subscription.drop_duplicates().dropna().reset_index(drop=True)
df__users_subscription


In [ ]:
df_users_labels = df_users_labels.merge(df__users_subscription,on=['dtEvent','customUserId'],how='left')
df_users_labels

In [ ]:
df_users_labels.to_csv("/Users/l.sergeev/Downloads/df_users_labels_test.csv")

In [ ]:
test = new_version_users_tracks_parsed_cat.loc[new_version_users_tracks_parsed_cat['customUserId'] == 'vk444438663']
test.to_csv("/Users/l.sergeev/Downloads/test_vk444438663.csv")

In [ ]:
new_version_users_tracks_parsed_cat.loc[new_version_users_tracks_parsed_cat['customUserId'] == 'vk469511292']

In [ ]:
test_group = new_version_users_tracks_parsed_cat[["dtEvent","customUserId",'subscription_type','idAppVersionTitle','method']][new_version_users_tracks_parsed_cat['subscription_type'] == 'none']


In [ ]:
test_group2 = test_group[test_group['method'] == 'cache']
test_group2.drop_duplicates().dropna().reset_index(drop=True)

In [ ]:
test_group2 = test_group2.drop_duplicates().dropna().reset_index(drop=True)

In [ ]:
test_group2.to_csv("/Users/l.sergeev/Downloads/test_group2.csv")

In [ ]:
test_group2